In [1]:
import os 
import sys

In [2]:
sys.path.append('/global/common/software/act/iabril/python/DustNGwBBP/')
os.chdir('/global/common/software/act/iabril/python/DustNGwBBP/')

In [76]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.colors as colors
from astropy.io import fits
import healpy as hp
import sacc
import pymaster as nmt
from scipy import stats

rcParams['font.size']=20
rcParams['text.usetex'] = True
rcParams['axes.linewidth']=1.5
rcParams['xtick.major.width']=1.5
rcParams['xtick.minor.width']=1.5
rcParams['ytick.major.width']=1.5
rcParams['ytick.minor.width']=1.5

In [11]:
from utils.sed import get_convolved_seds, get_band_names
from utils.bandpowers import get_ell_arrays
from dustngwbbp.compute_cl import get_windows, import_bandpasses
import utils.noise_calc as nc
from utils.params import LMIN, DELL, NBANDS, POLARIZATION_cov
from pte.sfid_class_pte import SfClass

band_names = get_band_names()
LMAX, LARR_ALL, LBANDS, LEFF = get_ell_arrays(LMIN, DELL, NBANDS)

nfreqs = len(band_names)
nmodes = len(POLARIZATION_cov)
nmaps = nfreqs * nmodes
ncross = (nmaps * (nmaps + 1)) // 2

indices_tr = np.triu_indices(nfreqs)
ncombs = len(indices_tr[0])
assert ncross == ncombs

NCOMP = 3
TYPE_COV = 'dfwt'
WEIGHT = 'Cl'

# Compute Q matrix

In [10]:
counter = np.arange(len(indices_tr[0]))
dicttr = {}
for i, counter_val in enumerate(counter):
    dicttr[(indices_tr[0][i], indices_tr[1][i])] = counter_val

In [12]:
fsky = nc.get_fsky()
bpss = import_bandpasses()
S = get_convolved_seds(band_names, bpss)

In [30]:
S.shape

(3, 6)

In [14]:
sens=2
knee=1
ylf=1

nell=np.zeros([nfreqs,LMAX+1])
_,nell[:,2:],_=nc.Simons_Observatory_V3_SA_noise(sens,knee,ylf,fsky,LMAX+1,1, atm_noise = True)

windows = get_windows(WEIGHT)
n_bpw=np.sum(nell[:,None,:]*windows[None,:,:],axis=2)
noise_array = n_bpw

In [17]:
print(noise_array)
print(noise_array.shape)

[[1.49640231e-04 2.24445582e-04 4.44449427e-04 1.11823579e-03
  3.55485442e-03 1.42570769e-02 7.20936104e-02 4.59490068e-01
  3.69033596e+00]
 [5.78141105e-05 6.78799083e-05 9.32607530e-05 1.44179996e-04
  2.49410525e-04 4.82056787e-04 1.04048464e-03 2.50745871e-03
  6.74599803e-03]
 [7.90956618e-07 6.93757034e-07 7.19157446e-07 7.83554235e-07
  8.81604702e-07 1.01970672e-06 1.21060634e-06 1.47428648e-06
  1.84112721e-06]
 [9.39113516e-07 8.10695939e-07 8.08936912e-07 8.28733468e-07
  8.59731762e-07 9.00424363e-07 9.51191622e-07 1.01315397e-06
  1.08793929e-06]
 [4.94965896e-06 3.42490345e-06 3.23101290e-06 3.20783674e-06
  3.23511845e-06 3.28756972e-06 3.35804060e-06 3.44441356e-06
  3.54638508e-06]
 [3.54409882e-05 2.16163478e-05 1.97193567e-05 1.93039641e-05
  1.92853262e-05 1.94325477e-05 1.96770842e-05 1.99947294e-05
  2.03765609e-05]]
(6, 9)


In [24]:
invnoise_ell = np.zeros((NBANDS, nfreqs, nfreqs)) + np.nan

for i in range(NBANDS):
    invnoise_ell[i,:,:]  = np.linalg.inv( np.diag( noise_array[:,i] ) )  # nfreq x nfreq

In [29]:
A_ell = np.zeros(( NBANDS, NCOMP, NCOMP)) + np.nan
B_ell = np.zeros(( NBANDS, NCOMP, nfreqs)) + np.nan
for i in range(NBANDS):
    A_ell[i,:,:] = np.matmul( S, np.matmul(invnoise_ell[i], S.transpose()) )
    B_ell[i,:,:] = np.matmul( S, invnoise_ell[i] )

In [31]:
Q_ell = np.zeros(( NBANDS, nfreqs )) + np.nan
for i in range(NBANDS):
    for j in range(nfreqs):
        Q_ell[i,j] = sum(A_ell[i,0,:] * B_ell[i,:,j])

In [33]:
Q_ell.shape

(9, 6)

In [35]:
save_path = '/global/cfs/cdirs/act/data/iabril/BBPower/230503_pte/'
np.savetxt(save_path + 'Qmatrix_clean.txt', Q_ell)

# Clean covs

In [36]:
Q_ell = np.loadtxt(save_path + 'Qmatrix_clean.txt')

In [46]:
covs_path = save_path + '0/'
covs_name = 'so_256_w2_p353_30_9_30_soflat_5.0_0.4_10_B_dcs_Cl_' # w_tot.fits

In [47]:
Cov_g = sacc.Sacc.load_fits(covs_path + 'w/' + covs_name + 'w_tot.fits').covariance.covmat
Cov_ng = sacc.Sacc.load_fits(covs_path + 'dfwt/' + covs_name + 'dfwt_tot.fits').covariance.covmat

In [49]:
Cov_g  = Cov_g.reshape([ncross, NBANDS, ncross, NBANDS])
Cov_ng = Cov_ng.reshape([ncross, NBANDS, ncross, NBANDS])

In [50]:
cov_cleaned_g = np.zeros((NBANDS, NBANDS))
cov_cleaned_ng = np.zeros((NBANDS, NBANDS))

for l1 in range(NBANDS): 
    for l2 in range(NBANDS):
        sumag = 0
        sumang = 0
        for n1 in range(nfreqs):
            for n2 in range(nfreqs):
                cros1 = (n1,n2)
                cros1_val = dicttr[tuple(sorted(cros1))]
                for n3 in range(nfreqs):
                    for n4 in range(nfreqs):
                        cros2 = (n3,n4)
                        cros2_val = dicttr[tuple(sorted(cros2))]
                        qval = (Q_ell[l1, n1] * Q_ell[l1, n2] * Q_ell[l2, n3] * Q_ell[l2, n4]) 
                        sumag  += qval * Cov_g[ cros1_val, l1, cros2_val, l2]
                        sumang += qval * Cov_ng[cros1_val, l1, cros2_val, l2]
        cov_cleaned_g[l1, l2] = sumag
        cov_cleaned_ng[l1,l2] = sumang

In [53]:
np.savetxt(save_path + 'Cov_g_clean.txt', cov_cleaned_g)
np.savetxt(save_path + 'Cov_ng_clean.txt', cov_cleaned_ng)

# Clean best fit and sampled cells

In [65]:
def clean_cells(Q, input_path, output_path):
    
    cell_sampled = sacc.Sacc.load_fits(input_path).mean
    cell_sampled = cell_sampled.reshape((ncross, NBANDS))
    
    # reshape into cell(nu, nuprime) shape
    cell_nunup_ell = np.zeros((NBANDS, nfreqs, nfreqs))
    for i in range(NBANDS):
        # https://stackoverflow.com/questions/17527693/transform-the-upper-lower-triangular-part-of-a-symmetric-matrix-2d-array-into
        X = np.zeros((nfreqs, nfreqs))
        X[np.triu_indices(X.shape[0], k = 0)] = cell_sampled[:,i]
        X = X + X.T - np.diag(np.diag(X))
        cell_nunup_ell[i] = X
        
    # clean:
    clean_cell = np.zeros(NBANDS) + np.nan
    for i in range(NBANDS):
        clean_cell[i] = np.matmul( (Q[i,:]).transpose(), np.matmul(cell_nunup_ell[i], Q[i,:]) )
    
    np.savetxt(output_path, clean_cell)

In [66]:
# check sampled same w and dfwt
path = save_path + f'0/'
path1 = covs_path + 'w/' + covs_name + 'w_tot.fits'
path2 = covs_path + 'dfwt/' + covs_name + 'dfwt_tot.fits'

In [67]:
a1 = sacc.Sacc.load_fits(path1)
a2 = sacc.Sacc.load_fits(path2)

In [68]:
np.all(a1.mean == a2.mean)

True

In [69]:
# only clean w/ sampled
for i in range(10):
    covs_path = save_path + f'{i}/'
    sample_cell_path = covs_path + 'w/' + covs_name + 'w_tot.fits'
    
    clean_cells(Q_ell, sample_cell_path, covs_path + 'clean_sample_cell.txt')
    

In [70]:
# same for cells_model:
for i in range(10):
    for wt in ['w', 'dfwt']:
        cells_path = f'/global/cfs/cdirs/act/data/iabril/BBPower/230503_pte/{i}/{wt}/'
        
        clean_cells(Q_ell, cells_path + 'cells_model.fits', cells_path + 'clean_cellbestfit.txt')

In [71]:
# proof from clean_cmb_david.ipynb
# celss_path = '/global/cfs/cdirs/act/data/iabril/BBPower/230503_pte/1/dfwt/cells_model.fits'
# a = sacc.Sacc.load_fits(celss_path)
# mean = a.mean
# array_mean = mean.reshape((ncross, NBANDS))#, order = 'F')
# nfreqs = 6
# cell_nunup_ell = np.zeros((NBANDS, nfreqs, nfreqs))
# for i in range(NBANDS):
#     # https://stackoverflow.com/questions/17527693/transform-the-upper-lower-triangular-part-of-a-symmetric-matrix-2d-array-into
#     X = np.zeros((nfreqs, nfreqs))
#     X[np.triu_indices(X.shape[0], k = 0)] = array_mean[:,i]
#     X = X + X.T - np.diag(np.diag(X))
#     cell_nunup_ell[i] = X
# clean_cell = np.zeros(NBANDS) + np.nan
# for i in range(NBANDS):
#     clean_cell[i] = np.matmul( (Q_ell[i,:]).transpose(), np.matmul(cell_nunup_ell[i], Q_ell[i,:]) )

# Compute chi2

In [73]:
nsims = 10

In [74]:
cov_g = np.loadtxt(save_path + 'Cov_g_clean.txt')
invcov_g = np.linalg.solve(cov_g, np.identity(len(cov_g)))
cov_ng = np.loadtxt(save_path + 'Cov_ng_clean.txt')
invcov_ng = np.linalg.solve(cov_ng, np.identity(len(cov_ng)))

invcovs = [invcov_g, invcov_ng]

chi2_array = np.zeros((2, nsims))

for i in range(nsims):
    
    path = save_path + f'{i}/'

    for j, wt in enumerate(['w', 'dfwt']):
        
        cdata  =  np.loadtxt(path + 'clean_sample_cell.txt')
        cmodel =  np.loadtxt(path + wt + '/clean_cellbestfit.txt')
        
        deltax = cdata - cmodel
        chi2_array[j, i] = np.linalg.multi_dot([deltax, invcovs[j], deltax])
        
        

In [75]:
chi2_array

array([[ 7.28867898, 16.22488159, 11.3836165 , 12.57814045, 17.85504313,
         7.97949668,  7.72997051, 14.52013072,  6.75885903, 12.20908263],
       [ 6.98309335, 15.82722805, 11.37787103, 12.30974277, 16.60028009,
         7.81176249,  7.23842949, 14.46681686,  6.69473757, 12.04453278]])

In [82]:
pval = stats.chi2.sf(chi2_array, df = NBANDS)

In [83]:
pval

array([[0.60709141, 0.06233141, 0.25032742, 0.1826422 , 0.03689303,
        0.53621378, 0.56157106, 0.10498888, 0.66220928, 0.20177726],
       [0.63887972, 0.07057836, 0.25069415, 0.19640535, 0.05535612,
        0.55322184, 0.61231069, 0.1066617 , 0.66886725, 0.21081039]])

In [ ]:
# save if it takes loads of time:
np.savetxt(save_path + f'chi2_array_g_{nsims}.txt', chi2_array[0])
np.savetxt(save_path + f'chi2_array_ng_{nsims}.txt', chi2_array[1])
